In [82]:
import requests as req

from bs4 import BeautifulSoup as bs

import pandas as pd

## Ejemplo Terremotos

In [2]:
url='https://www.emsc-csem.org/Earthquake/'

In [3]:
html=req.get(url).content

In [4]:
sopa=bs(html, 'html.parser')

In [9]:
tablas=sopa.find_all('table')

In [30]:
tabla=tablas[3]

type(tabla)

bs4.element.Tag

In [29]:
cabeceras=tabla.find('thead').find_all('th')

type(cabeceras)

bs4.element.ResultSet

In [26]:
cabezas=[]

for e in cabeceras:
    
    cabezas.append(e.text)

In [43]:
cabeceras=[e.text for e in tabla.find('thead').find_all('th')][1:-1]

cabeceras

['Date & Time UTC',
 'Latitude degrees',
 'Longitude degrees',
 'Depth km',
 'Mag  [+]',
 'Region name  [+]']

In [32]:
body=tabla.find('tbody')

filas=body.find_all('tr')  # filas de la tabla

In [37]:
[e for e in filas[0].text.split('\xa0') if e!='']

['earthquake2022-06-22',
 '07:07:12.603min ago19.15',
 'N',
 '155.46',
 'W',
 '32Md2.2',
 'ISLAND OF HAWAII, HAWAII2022-06-22 07:10']

In [44]:
[e.text.strip() for e in filas[1].find_all('td') if e.text.strip()!='']

['earthquake2022-06-22\xa0\xa0\xa007:00:34.010min ago',
 '7.33',
 'S',
 '129.50',
 'E',
 '165',
 'M',
 '4.6',
 'KEPULAUAN BABAR, INDONESIA',
 '2022-06-22 07:10']

In [65]:
fila_uno=[e.text.strip() for e in filas[0].find_all('td') if e.text.strip()!='']


fila_uno_prima=[]

for i,e in enumerate(fila_uno):
    
    limpio=e.replace('earthquake', '').replace('\xa0', ' ')
    
    junto=''
    
    if i==1 or i==3:
        junto=fila_uno[i]+' '+fila_uno[i+1]
        
        fila_uno_prima.append(junto)
        
        continue
        
    
    fila_uno_prima.append(limpio)

    
fila_uno_prima=fila_uno_prima[:-1]

fila_uno_prima

['2022-06-22   07:07:12.603min ago',
 '19.15 N',
 'N',
 '155.46 W',
 'W',
 '32',
 'Md',
 '2.2',
 'ISLAND OF HAWAII, HAWAII']

In [66]:
lst=[2, 3, 4]

for e in lst:
    fila_uno_prima.pop(e)

In [67]:
fila_uno_prima

['2022-06-22   07:07:12.603min ago',
 '19.15 N',
 '155.46 W',
 '32',
 '2.2',
 'ISLAND OF HAWAII, HAWAII']

In [84]:
tabla=sopa.find_all('tr', class_= 'normal')

data=[]

for tr in tabla:
    
    fecha=tr.contents[3].find('a').text
    
    latitud=tr.contents[4].text + tr.contents[5].text
    longitud=tr.contents[6].text + tr.contents[7].text
    
    profundidad=tr.contents[8].text
    magnitud=tr.contents[10].text
    
    region=tr.contents[11].text
    
    data.append([fecha, latitud, longitud, region, profundidad, magnitud])
    
    
terremotos=pd.DataFrame(data, columns=['fecha', 'latitud', 'longitud', 'region', 'profundidad', 'magnitud'])

terremotos.head()

,fecha,latitud,longitud,region,profundidad,magnitud
0,2022-06-22 07:07:12.6,19.15 N,155.46 W,"ISLAND OF HAWAII, HAWAII",32,2.2
1,2022-06-22 07:00:34.0,7.33 S,129.50 E,"KEPULAUAN BABAR, INDONESIA",165,4.6
2,2022-06-22 06:49:12.9,2.72 N,128.78 E,"HALMAHERA, INDONESIA",40,4.8
3,2022-06-22 06:48:09.2,2.08 N,96.49 E,"SIMEULUE, INDONESIA",21,4.9
4,2022-06-22 06:38:42.0,11.58 N,86.82 W,NEAR COAST OF NICARAGUA,34,2.8
